This is a tweet scaping script based *Heavily* on the example script found here: https://gist.github.com/bjmarsh/315a632aa1ab0e8436e631f8a1acf40b orignially created by Bennett Marsh.

In [1]:
from collections import defaultdict
import os, sys
import time
import pandas as pd
import GetOldTweets3 as got


In [3]:
os.makedirs('tweet_data', exist_ok=True)
users = ["elonmusk"]
username = users[0]

In [18]:
count = 10
# Creation of query object                                                                                                                                                                                      
tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                               .setMaxTweets(count)\
                                               .setSince("2020-05-30")\
                                               .setUntil("2020-05-31")
tweets = None
for ntries in range(2):        
    try:
        tweets = got.manager.TweetManager.getTweets(tweetCriteria)
    except SystemExit:
        print("Trying again in 15 minutes.")
        time.sleep(15*60)
    else:
        break
if tweets is None:
    print("Failed after 2 tries, quitting!")
    exit(1)

In [19]:
len(tweets)

4

In [9]:
tweets[0]

Got it, the getTweets() function returns a tweet object.  No docstring on the got tweet object though.

In [12]:
tweets[0].id
tweets[0].to

'NASASpaceflight'

Bennet's original script does just fine in gathering up all of Elon's tweets.  I'd like to have a record of the semantic content of the tweet/reply conversations that Elon has with his followers and twitters API does not make this properly available.  I'm going to have to fudge it, but I think that this algorithm will do a decent job of getting at least some of the conversations Elon has.

In [20]:
def get_other_user_reply(username,t_init,t_final): 
    #searches a secondary user's tweets within a range of time and 
    #returns tweets that either reply to or @elonmusk
    print(username)
    count = 0
    # Creation of query object                                                                                                                                                                                      
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
                                               .setMaxTweets(count)\
                                               .setSince(t_init)\
                                               .setUntil(t_final)
    # Creation of list that contains all tweets                                                                                                                                                                     
    tweets = None
    for ntries in range(5):
        try:
            tweets = got.manager.TweetManager.getTweets(tweetCriteria)
        except SystemExit:
            print("Trying again in 15 minutes.")
            time.sleep(15*60)
        else:
            break
    if tweets is None:
        print("Failed after 5 tries, quitting!")
        exit(1)

    data = defaultdict(list)
    for t in tweets:
        if t.to == 'elonmusk' or t.mentions == '@elonmusk':
            data["username"].append(username)
            data["tweet_id"].append(t.id)
            data["reply_to"].append(t.to)
            data["date"].append(t.date)
            data["retweets"].append(t.retweets)
            data["favorites"].append(t.favorites)
            data["hashtags"].append(list(set(t.hashtags.split())))
            data["mentions"].append(t.mentions)
            data["text"].append(t.text)
            data["permalink"].append(t.permalink)
        else:
            pass
    return data

In [21]:
elon_tweets_df = pd.read_csv('./elonmusk.csv')

In [24]:
elon_tweets_df.columns

Index(['Unnamed: 0', 'username', 'tweet_id', 'reply_to', 'date', 'retweets',
       'favorites', 'hashtags', 'mentions', 'text', 'permalink'],
      dtype='object')